# Algorithms for market basket analysis.

Keep this cell updating till updating till we complete all the algorithms.

## Algorithm 1: Apriori algorithm

3 key metrics required

*  Support: frquency of an item in the itemset
*  Confidence: Conditional probability of consquent being bought if antescent is being bought.
*  Lift: Confidence(Both antesent and consequent)/Support(consequent)

Final goal to get this algorithm to work on instacart's market basket analysis.

### Psuedocode

Pseudo-code:

Ck: Candidate itemset of size k

Lk : frequent itemset of size k

L1 = {frequent items};

    for (k = 1; Lk !=∅; k++) do begin

     Ck+1 = candidates generated from Lk; for each transaction t in database do
 
     increment the count of all candidates in Ck+1 that are contained in t
 
     Lk+1 = candidates in Ck+1 with min_support end
 
    return ∪k Lk; 

### Import requirements
numpy
pandas
From mlxtend import apriori, association rules and, fp_growth. 

In [3]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth 
import warnings
warnings.filterwarnings('ignore')
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [4]:
orders_products = pd.read_csv("/kaggle/input/instacart-customer-data/order_products__prior.csv/order_products__prior.csv")
orders_products.info()#(memory_usage = "deep")
#11.2 GBs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


In [5]:
chunksize = 2500
min_support = 0.005

# Store results across batches
all_frequent_itemsets = []

for chunk in pd.read_csv("/kaggle/input/orders-products/orders_products", chunksize=chunksize):
    #chunk = chunk.dropna(subset=["product_name"])  # Drop missing product names
    chunk["product_name"] = chunk["product_name"].astype(str)  # Ensure all names are strings

    # Group products by order
    transactions = chunk.groupby("order_id")["product_name"].apply(list).tolist()

    # Use TransactionEncoder with sparse matrix to reduce memory usage
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions, sparse=True)
    df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

    # Apply Apriori algorithm on the batch
    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)
    all_frequent_itemsets.append(frequent_itemsets)

# Combine all frequent itemsets from different batches
final_frequent_itemsets = pd.concat(all_frequent_itemsets).groupby("itemsets").sum().reset_index()

KeyboardInterrupt: 

The above given method works well for small datasets but in the case, we have 32 million entries and this'll take too much time. Also, even with batch processing, the code crashes out after some time. So, why not try other method with python generators.

In [6]:
orders = orders_products.set_index('order_id')['product_id'].rename('item_id')
orders

order_id
2          33120
2          28985
2           9327
2          45918
2          30035
           ...  
3421083    39678
3421083    11352
3421083     4600
3421083    24852
3421083     5020
Name: item_id, Length: 32434489, dtype: int64

### Helper functions

1) freq: calculates frequency for items
2) order_count: Returns number of unique orders
3) get_item_pairs: it will give us item pairs in 2
4) merge_item_stats: Returns frequency and support associated with item
5) merge_item_name: Returns name associated with item


In [7]:
#Here, if it's a series, we count via value_counts and if it's not a series, we convert and count it with counter 
def freq(iterable):
    if type(iterable) == pd.core.series.Series:  
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

In [8]:
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))

In [9]:
#This generator function will get us item pairs from the orders column
def get_item_pairs(order_item, num_items=2):
    order_item = order_item.reset_index().values # Drop = true prevents us from getting an extra column
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]

        for item_pair in combinations(item_list,num_items):
            yield item_pair

# df = pd.DataFrame({
#     'order_id': [1, 1, 1, 2, 2],
#     'item': ['apple', 'egg', 'milk', 'egg', 'milk']
# })

# pairs = list(get_item_pairs(df, num_items=2))
# print(pairs)
print(next(get_item_pairs(orders)))

(33120, 28985)


In [10]:
# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))

In [11]:
# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]

### Association rules function

In [12]:
def association_rules(order_item, min_support):
    print("Starting order_item: {:22d}".format(len(order_item)))

    #Calculate the item frequency and support
    item_stats = freq(order_item).to_frame('freq')
    item_stats['support'] = item_stats['freq']/order_count(order_item) * 100

    # Filter from order_item items below min support 
    qualifying_items = item_stats[item_stats['support']>=min_support].index
    order_id = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))

    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))

    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)

    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))

    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))

    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [13]:
%%time 
rules = association_rules(orders, 0.01)

Starting order_item:               32434489
Items with support >= 0.01:           10906
Remaining order_item:              32434489
Remaining orders with 2+ items:     3058126
Remaining order_item:              32277741
Item pairs:                        54110893
Item pairs with support >= 0.01:      47826

CPU times: user 10min 9s, sys: 19.7 s, total: 10min 28s
Wall time: 10min 28s


In [15]:
item_name   = pd.read_csv('/kaggle/input/instacart-customer-data/products.csv/products.csv')
item_name   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('freqAB', ascending=False)
display(rules_final)

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
15551,Bag of Organic Bananas,Organic Strawberries,40029,1.308939,376676,12.317216,263563,8.618448,0.106269,0.151876,0.012330
27093,Banana,Organic Strawberries,38564,1.261034,470518,15.385828,263563,8.618448,0.081961,0.146318,0.009510
10387,Bag of Organic Bananas,Organic Hass Avocado,37628,1.230427,376676,12.317216,212887,6.961355,0.099895,0.176751,0.014350
26615,Banana,Organic Baby Spinach,35590,1.163785,470518,15.385828,240755,7.872632,0.075640,0.147827,0.009608
14842,Banana,Organic Avocado,34032,1.112838,470518,15.385828,176303,5.765067,0.072329,0.193031,0.012546
...,...,...,...,...,...,...,...,...,...,...,...
26758,Organic Ketchup,Organic Tomato Cluster,306,0.010006,15230,0.498017,64164,2.098148,0.020092,0.004769,0.009576
2667,Organic Whole String Cheese,Organic Mixed Berry Yogurt & Fruit Snack,306,0.010006,59607,1.949135,5555,0.181647,0.005134,0.055086,0.028262
10318,Fridge Pack Cola,Granny Smith Apples,306,0.010006,18076,0.591081,35988,1.176799,0.016929,0.008503,0.014385
34776,Organic Baby Broccoli,100% Raw Coconut Water,306,0.010006,31712,1.036975,37563,1.228301,0.009649,0.008146,0.007856
